In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# !pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
import requests
import os

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [ ]:
weather_api_key = os.getenv("WEATHER_API_KEY")

In [7]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={weather_api_key}&query={city}'

  response = requests.get(url)

  return response.json()

In [51]:
@tool
def temp_converter(celsius: str) -> str:
    """
    This function converts Celsius temperature to Fahrenheit 
    """
    
    print("\n\n", celsius, "\n\n")
    
    # Clean the input - remove any non-numeric characters except decimal point and minus sign
    import re
    celsius_clean = re.sub(r'[^\d.-]', '', celsius)
    
    try:
        fahrenheit = (float(celsius_clean) * 9/5) + 32
        return f"{float(celsius_clean)}°C is equal to {fahrenheit:.1f}°F"
    except ValueError:
        return f"Error: Could not convert '{celsius}' to a temperature value"

In [43]:
temp_converter.invoke({'celsius': "29"})

'29.0°C is equal to 84.2°F'

In [11]:
chat_llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation"
)
model = ChatHuggingFace(llm=chat_llm)

In [12]:
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_classic import hub

In [13]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt

In [52]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=model,
    tools=[search_tool, get_weather_data, temp_converter],
    prompt=prompt
)

In [47]:
# agent

In [57]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data, temp_converter],
    verbose=True
)

In [58]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Maharashtra, then find it's current temperature in celcius. And convert it to Fahrenheit"})
print(response)



> Entering new AgentExecutor chain...
I need to find the capital of Maharashtra first.
Action: duckduckgo_search
Action Input: capital of Maharashtra
Observ28 Jun 2025 · Mumbai also known as Bombay is the capital city of the Indian state of Maharashtra. Mumbai is the financial capital and the most populous city proper of ... Religion in Maharashtra · List of tallest buildings in... · Economy of Mumbai · Harbor 14 Jul 2025 · Mumbai is the capital of Maharashtra and the most populous urban area in India, with Nagpur serving as the winter capital. The Godavari and Krishna are ... Maharashtra Legislative... · Economy of Maharashtra · Maharashtra Legislature 15 Apr 2025 · Mumbai, the capital of Maharashtra and often described as the New York of India or Manhattan of India, is the financial capital and the most populous ... 1 Apr 2025 · NAINA, Maharashtra's upcoming smart city, can draw insights from China's Guiyang and emerge as India's science data capital. 2 Apr 2025 · Maharashtra's cap

In [23]:
display(response['output'])

'The capital of Maharashtra is Mumbai. The current weather in Mumbai is smoke with a temperature of 29°C, wind speed of 17 km/h from the direction WNW, and humidity at 43%. There is no precipitation currently.'

In [24]:
import textwrap
output = response['output']
print(textwrap.fill(output, width=200))

The capital of Maharashtra is Mumbai. The current weather in Mumbai is smoke with a temperature of 29°C, wind speed of 17 km/h from the direction WNW, and humidity at 43%. There is no precipitation
currently.
